In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-179267
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-179267


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "Training-PMZ"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": choice(0.1, 1, 1.4),
        "--max_iter": choice(60, 80, 100, 120, 140)
    })

# Specify a Policy
policy = BanditPolicy(slack_factor=0.05, evaluation_interval=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=12,
                             max_concurrent_runs=4)    

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_1dd13935-4d5b-4f57-a057-21dec21450ed
Web View: https://ml.azure.com/runs/HD_1dd13935-4d5b-4f57-a057-21dec21450ed?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-179239/workspaces/quick-starts-ws-179239&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-27T21:31:01.221494][API][INFO]Experiment created<END>\n""<START>[2022-01-27T21:31:02.115245][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-01-27T21:31:02.784620][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_1dd13935-4d5b-4f57-a057-21dec21450ed
Web View: https://ml.azure.com/runs/HD_1dd13935-4d5b-4f57-a057-21dec21450ed?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-179239/workspaces/quick-starts-ws-179239&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_1dd13935-4d5b-4f57-a057-21dec21450ed',
 'target': 'Training-PMZ',
 'status': 'Completed',
 'startTimeUtc': '2022-01-27T21:31:00.972522Z',
 'endTimeUtc': '2022-01-27T21:53:44.590734Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4f3ace71-cc00-493e-8c98-94332cb5ec71',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1064-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.37.0',
  'space_size': '15',
  'score': '0.9118079823432039',
  'best_child_run_id': 'HD_1dd13935-4d5b-4f57-a057-21dec21450ed_6',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg179239.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_1dd13935-4d5b-4f57-a057-21dec21450ed/azureml-

In [16]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_filenames = best_run.get_file_names()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(f"Details:  {best_run.get_details()}")
print(f"Metrics:  {best_run_metrics}")
print(f"Files:    {best_run_filenames}")
print(f"Run Id:   {best_run.id}")
      
joblib.dump(best_run, './model.joblib')

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])

Details:  {'runId': 'HD_1dd13935-4d5b-4f57-a057-21dec21450ed_6', 'target': 'Training-PMZ', 'status': 'Completed', 'startTimeUtc': '2022-01-27T21:49:41.006553Z', 'endTimeUtc': '2022-01-27T21:50:10.100105Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '4f3ace71-cc00-493e-8c98-94332cb5ec71', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '1', '--max_iter', '80'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'Training-PMZ', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'sklearn-env', 'ver

TypeError: can't pickle _thread.RLock objects

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
src = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds_jupyter = TabularDatasetFactory.from_delimited_files(src)

In [4]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds_jupyter)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
train_data = pd.concat([x_train, y_train], axis=1)
train_data.to_csv('training_pandas.csv')
ds_train = ws.get_default_datastore()
ds_train.upload(src_dir='.', target_path='training_data')
train_ds = Dataset.Tabular.from_delimited_files(path=[(ds_train, ('training_data/training_pandas.csv'))])

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 8 files
Uploading ./CODEOWNERS
Uploaded ./CODEOWNERS, 1 files out of an estimated total of 8
Uploading ./conda_dependencies.yml
Uploaded ./conda_dependencies.yml, 2 files out of an estimated total of 8
Uploading ./README.md
Uploaded ./README.md, 3 files out of an estimated total of 8
Uploading ./train.py
Uploaded ./train.py, 4 files out of an estimated total of 8
Uploading ./training_pandas.csv
Uploaded ./training_pandas.csv, 5 files out of an estimated total of 8
Uploading ./udacity-project.ipynb
Uploaded ./udacity-project.ipynb, 6 files out of an estimated total of 8
Uploading ./.ipynb_checkpoints/udacity-project-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/udacity-project-checkpoint.ipynb, 7 files out of an estimated total of 8
Uploading ./__pycache__/train.cpython-36.pyc
Uploaded ./__pycache__/train.cpython-36.pyc, 8 files out of an estimated total of 8
Uploaded 8 files


In [5]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=3,
    compute_target=cpu_cluster,
    enable_early_stopping = True,
    enable_onnx_compatible_models = True)

In [6]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on Training-PMZ with default configuration
Running on remote compute: Training-PMZ


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ad2920ef-e853-4f4d-8f24-ba540136cdd1,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [7]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best, automl_model = automl_run.get_output()
automl_best.get_metrics()

{'balanced_accuracy': 0.7461542708623456,
 'average_precision_score_macro': 0.8221273050882756,
 'precision_score_weighted': 0.9098686140495458,
 'f1_score_micro': 0.9161533313872917,
 'recall_score_macro': 0.7461542708623456,
 'norm_macro_recall': 0.49230854172469124,
 'log_loss': 0.2074839061993019,
 'AUC_macro': 0.9467393475270747,
 'average_precision_score_micro': 0.9812473835893761,
 'recall_score_weighted': 0.9161533313872917,
 'AUC_weighted': 0.9467393239410818,
 'AUC_micro': 0.9805213028034951,
 'weighted_accuracy': 0.9583491309236106,
 'f1_score_macro': 0.7690115747303018,
 'precision_score_macro': 0.7992403702792288,
 'recall_score_micro': 0.9161533313872917,
 'matthews_correlation': 0.542751782922335,
 'f1_score_weighted': 0.9120752313505038,
 'precision_score_micro': 0.9161533313872917,
 'accuracy': 0.9161533313872917,
 'average_precision_score_weighted': 0.954712446258231,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_ad2920ef-e853-4f4d-8f24-ba540136cdd1